# 获取表单数据

In [ ]:
# 要使用表单，首先安装 python-multipart 。
$ pip install python-multipart

In [ ]:
# 导入 Form
from fastapi import FastAPI, Form

In [ ]:
# 定义 Form 参数
# 要声明表单体，您需要使用 Form 显式地，因为如果没有它，参数将被解释为查询参数或正文（JSON）参数。
@app.post("/login/")
async def login(username: Annotated[str, Form()], password: Annotated[str, Form()]):

In [ ]:
# 表单数据通常使用“媒体类型” application/x-www-form-urlencoded 进行编码。
# 但当表单包含文件时，它会被编码为 multipart/form-data 。

In [ ]:
# Pydantic 模型用于表单
# FastAPI 将从请求中的表单数据中提取每个字段的值，并返回你定义的 Pydantic 模型。
class FormData(BaseModel):
    username: str
    password: str

@app.post("/login/")
async def login(data: Annotated[FormData, Form()]):

In [ ]:
# 禁止额外表单字段
# 仅限制表单字段为 Pydantic 模型中声明的字段，并禁止任何额外字段。(自 FastAPI 版本 0.114.0 起支持)
# 使用 Pydantic 的模型配置来 forbid 任何 extra 字段：
class FormData(BaseModel):
    username: str
    password: str
    model_config = {"extra": "forbid"}

@app.post("/login/")
async def login(data: Annotated[FormData, Form()]):# 如果尝试发送一些额外数据，他们将收到一个错误响应。
# 例如，如果尝试发送表单字段：
# username: Rick
# password: Portal Gun
# extra: Mr. Poopybutthole
# 他们将收到一个错误响应，告诉他们字段 extra 不允许

# 文件上传

In [ ]:
# 使用 File 、 bytes 和 UploadFile 声明请求中要上传的文件，将以表单数据的形式发送。

In [ ]:
# 要接收上传的文件，首先安装 python-multipart 。
$ pip install python-multipart
# 因为上传的文件以“表单数据”的形式发送。

In [ ]:
# 导入 File
from fastapi import FastAPI, File, UploadFile

In [ ]:
# 定义 File 参数
@app.post("/files/")
async def create_file(file: Annotated[bytes, File()]):
# File 是一个直接继承自 Form 的类。
# 要声明文件主体，需要使用 File ，因为否则参数将被解释为查询参数或主体（JSON）参数。
# 这些文件将以“表单数据”的形式上传。# 如果你将路径操作函数参数的类型声明为 bytes ，FastAPI 将为你读取文件，并将内容作为 bytes 返回。
# 请记住，这意味着整个内容将被存储在内存中。这对于小型文件来说效果很好。
# 但是，在某些情况下，您可能会从使用 UploadFile 中受益。

In [ ]:
# 文件参数 UploadFile
@app.post("/uploadfile/")
async def create_upload_file(file: UploadFile):
# 使用 UploadFile 比 bytes 有几个优点：
    # 您不需要在参数的默认值中使用 File() 。
    # 它使用“分页”文件：
        # 一个文件在内存中存储，直到达到最大大小限制，之后它将被存储在磁盘上。
    # 这意味着它能够很好地处理大型文件，如图像、视频、大型二进制文件等，而不会消耗所有内存。
    
    # 你可以从上传的文件中获取元数据。
    # 它有一个类似文件的 async 接口。
    # 它暴露了一个实际的 Python SpooledTemporaryFile 对象，你可以直接将其传递给其他期望文件对象库。

In [ ]:
# UploadFile类型

# UploadFile 具有以下属性：
    # filename : 一个带有原始文件名上传的文件（例如 myimage.jpg ）。
    # content_type : 一个 str 带有内容类型（MIME 类型/媒体类型）（例如 image/jpeg ）。
    # file : 一个 SpooledTemporaryFile （类似文件对象）。这是真正的 Python 文件对象，你可以直接将其传递给其他期望“类似文件”对象功能的函数或库。

# UploadFile 有以下 async 方法。它们都调用相应的文件方法（使用内部 SpooledTemporaryFile ）。
    # write(data) : 将 data ( str 或 bytes ) 写入文件。
    # read(size) : 读取 size ( int ) 字节/字符的文件。
    # seek(offset) : 跳转到文件中的字节位置 offset ( int )。
        # 例如， await myfile.seek(0) 将跳转到文件开头。
        # 这特别有用，如果你运行 await myfile.read() 一次，然后需要再次阅读内容。
    # close() : 关闭文件。
# 由于这些方法都是 async 方法，您需要等待它们。

# FastAPI 的 UploadFile 直接继承自 Starlette 的 UploadFile ，但添加了必要的组件以使其与 Pydantic 和 FastAPI 的其他部分兼容。

In [ ]:
# 获取file内容
# 在 async 路径操作函数内部，你可以通过以下方式获取内容：
contents = await myfile.read()

# 如果你处于一个正常的 def 路径操作函数内部，你可以直接访问 UploadFile.file ，例如：
contents = myfile.file.read()

In [ ]:
# 可选文件上传
@app.post("/files/")
async def create_file(file: Annotated[bytes | None, File()] = None):

@app.post("/uploadfile/")
async def create_upload_file(file: UploadFile | None = None):

In [ ]:
# 带有附加元数据的 UploadFile
# 使用 File() 与 UploadFile 一起使用，例如，设置额外的元数据：
@app.post("/files/")
async def create_file(file: Annotated[bytes, File(description="A file read as bytes")]):

@app.post("/uploadfile/")
async def create_upload_file(
    file: Annotated[UploadFile, File(description="A file read as UploadFile")],
):

In [ ]:
# 多文件上传
# 前端网页
@app.get("/")
async def main():
    content = """
<body>
<form action="/files/" enctype="multipart/form-data" method="post">
<input name="files" type="file" multiple>
<input type="submit">
</form>
<form action="/uploadfiles/" enctype="multipart/form-data" method="post">
<input name="files" type="file" multiple>
<input type="submit">
</form>
</body>
    """
    return HTMLResponse(content=content)

# 后端服务
@app.post("/files/")
async def create_files(files: Annotated[list[bytes], File()]):

@app.post("/uploadfiles/")
async def create_upload_files(files: list[UploadFile]):

# 前端接收
# 获得 list 数量的 bytes 或 UploadFile 。

In [ ]:
# 多文件上传及附加元数据

# 前端网页
@app.get("/")
async def main():
    content = """
<body>
<form action="/files/" enctype="multipart/form-data" method="post">
<input name="files" type="file" multiple>
<input type="submit">
</form>
<form action="/uploadfiles/" enctype="multipart/form-data" method="post">
<input name="files" type="file" multiple>
<input type="submit">
</form>
</body>
    """
    return HTMLResponse(content=content)

# 后端服务
@app.post("/files/")
async def create_files(
    files: Annotated[list[bytes], File(description="Multiple files as bytes")],
):
    return {"file_sizes": [len(file) for file in files]}

@app.post("/uploadfiles/")
async def create_upload_files(
    files: Annotated[
        list[UploadFile], File(description="Multiple files as UploadFile")
    ],
):
    return {"filenames": [file.filename for file in files]}

# 同时请求表单数据和文件

In [ ]:
# 你可以使用 File 和 Form 同时定义文件和表单字段。
# 要接收上传的文件和/或表单数据，首先安装 python-multipart 。
$ pip install python-multipart

In [ ]:
# 导入 File 和 Form
from fastapi import FastAPI, File, Form, UploadFile

In [ ]:
# 定义 File 和 Form 参数
@app.post("/files/")
async def create_file(
    file: Annotated[bytes, File()],
    fileb: Annotated[UploadFile, File()],
    token: Annotated[str, Form()],
):

# 其他版本和变种
# 如果可能的话，更倾向于使用 Annotated 版本。
# @app.post("/files/")
# async def create_file(
#     file: bytes = File(), fileb: UploadFile = File(), token: str = Form()
# ):

# 这些文件和表单字段将被作为表单数据上传，您将收到这些文件和表单字段。

In [ ]:
# 你可以将一些文件声明为 bytes ，将另一些文件声明为 UploadFile 。

In [ ]:
# 你可以在路径操作中声明多个 File 和 Form 参数，但你不能同时声明 Body 字段，你期望以 JSON 形式接收这些字段，因为请求的正文将使用 multipart/form-data 而不是 application/json 进行编码。
# 这不是 FastAPI 的限制，它是 HTTP 协议的一部分。